In [1]:
import pandas as pd
import os
from glob import glob
import numpy as np

# Assign directory
PATH = 'Top 100 Influencers' 
EXT = "*.csv"


all_csv_files = [
    file  # Just use the 'file' directly
    for path, subdir, files in os.walk(PATH)
    for file in glob(os.path.join(path, EXT))
]




In [2]:
#LOOK INTO MERGING COLUMNS without the header

# Read the first CSV file with headers
first_file = all_csv_files[0]
df = pd.read_csv(first_file, delimiter=',',  na_values=['N/A', '?', 'None'])  # Explicitly set the delimiter
headers = df.columns.tolist()  # Store the headers
# print(headers)
# df.head()


# Create an empty list to store the dataframes
df_merged= []
# Create a list to store channel names
channel_names = []

# Define the extract_channel function
def extract_channel(filename):
    try:
        return filename.split('_')[0]  # Extract the channel name from the filename
    except Exception as e:
        print(f"Error extracting channel from filename: {filename} - Error: {e}")
        return None

In [3]:

# Loop through all CSV files (including the first one)
for csv_file in all_csv_files:  # Process the first 10 [:10] files
    df = pd.read_csv(csv_file, header=None, delimiter=',', names=headers)
    channel_name = extract_channel(os.path.basename(csv_file))  # Extract channel name
    
    # Repeat the channel name for each row in the DataFrame (excluding the header)
    channel_names.extend([channel_name] * (len(df) - 1))
    
    df_merged.append(df.iloc[1:, :])  # Remove header and append


#without converting df_merged type list to dataframe
final_df = pd.concat(df_merged, ignore_index=True)

# Assign the channel names to the 'CHANNEL' column
final_df['CHANNEL'] = channel_names  # Assign the list of channel names

In [57]:




# Print unique values in 'CHANNEL' column and the final DataFrame
# print("Unique values in 'CHANNEL' column:", final_df['CHANNEL'].unique())


# Search for rows that contain 'kendalljenner' in any column (case-insensitive)

# Iterate through each CSV file
# for file in all_csv_files:
#     try:
#         # Load the dataset
#         df = pd.read_csv(file, header=None, delimiter=',', names=headers)

#         # Search for rows that contain 'kendalljenner' in any column (case-insensitive)
#         search_result = df[df.apply(lambda row: row.astype(str).str.contains('@brunamarquezine', case=False, na=False).any(), axis=1)]
# #         search_result = df[df.apply(lambda row: row.astype(str).str.fullmatch('zara', case=False, na=False).any(), axis=1)]
#         # Print the filename and search result
#         if not search_result.empty:
#             print(f"--- Results in file: {file} ---")
#             print(search_result)

#     except Exception as e:
#         print(f"Error reading file {file}: {e}")





In [58]:
# final_df.iloc[90:121]

In [4]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22638 entries, 0 to 22637
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   #                   22638 non-null  object
 1   NAME                22638 non-null  object
 2   FOLLOWERS           22552 non-null  object
 3   ER                  21962 non-null  object
 4   COUNTRY             22336 non-null  object
 5   TOPIC OF INFLUENCE  12356 non-null  object
 6   POTENTIAL REACH     22638 non-null  object
 7   CHANNEL             22638 non-null  object
dtypes: object(8)
memory usage: 1.4+ MB


In [5]:
#number of missing entries in each column

miss_values = final_df.isna().sum()
print(miss_values)

#                         0
NAME                      0
FOLLOWERS                86
ER                      676
COUNTRY                 302
TOPIC OF INFLUENCE    10282
POTENTIAL REACH           0
CHANNEL                   0
dtype: int64


In [6]:
# Remove rows with NaN values in 'FOLLOWERS' and COUNTRY
final_df.dropna(subset=['FOLLOWERS'], inplace=True)
# final_df.dropna(subset=['COUNTRY'], inplace=True)
# final_df.dropna(subset=['ER'], inplace=True)
final_df.dropna(subset=['TOPIC OF INFLUENCE'], inplace=True)
final_df = final_df.drop('#', axis=1)


In [7]:
miss_values2 = final_df.isna().sum()
print(miss_values2)

NAME                    0
FOLLOWERS               0
ER                     32
COUNTRY               194
TOPIC OF INFLUENCE      0
POTENTIAL REACH         0
CHANNEL                 0
dtype: int64


In [8]:
# hyphen_count = (final_df['ER'] == '-').sum()
# print(f"Number of hyphens in 'ER' column: {hyphen_count}")



In [9]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12295 entries, 32 to 22637
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   NAME                12295 non-null  object
 1   FOLLOWERS           12295 non-null  object
 2   ER                  12263 non-null  object
 3   COUNTRY             12101 non-null  object
 4   TOPIC OF INFLUENCE  12295 non-null  object
 5   POTENTIAL REACH     12295 non-null  object
 6   CHANNEL             12295 non-null  object
dtypes: object(7)
memory usage: 768.4+ KB


In [10]:

# Remove '%' and convert to numeric
# final_df['ER'] = final_df['ER'].str.replace('%', '', regex=False)  # Remove '%'
final_df['ER'] = pd.to_numeric(final_df['ER'].str.replace('%', '', regex=False), errors='coerce')



In [11]:
final_df.isna().sum()

NAME                     0
FOLLOWERS                0
ER                    2322
COUNTRY                194
TOPIC OF INFLUENCE       0
POTENTIAL REACH          0
CHANNEL                  0
dtype: int64

In [12]:
# Convert K,M,B to decimals 

from decimal import Decimal,InvalidOperation  # Import the Decimal class


def convert_followers(followers_str):
    """Converts a string with K, M, or B suffixes to a Decimal value."""
    try:
        followers_str = followers_str.upper()  # Standardize case
        if followers_str[-1] in ('K', 'M', 'B'):
            num, magnitude = followers_str[:-1], followers_str[-1]
            magnitude_multiplier = {'K': 10**3, 'M': 10**6, 'B': 10**9}
            return Decimal(num) * magnitude_multiplier[magnitude]
        else:
            return Decimal(followers_str)
    except (InvalidOperation, TypeError, IndexError) as e:
#         print(f"Error converting value: {text} - Error: {e}")  # Print the error message
        return None

    
# Apply function to have K,M,B decimals 

final_df['FOLLOWERS'] = final_df['FOLLOWERS'].apply(convert_followers)
final_df['POTENTIAL REACH'] = final_df['POTENTIAL REACH'].apply(convert_followers)



In [13]:
# Convert to FLOAT type FOLLOWERS and POTENTIAL REACH columns

final_df['FOLLOWERS'] = pd.to_numeric(final_df['FOLLOWERS'], errors='coerce')
final_df['POTENTIAL REACH'] = pd.to_numeric(final_df['POTENTIAL REACH'])

In [14]:
final_df.dropna(subset=['ER'], inplace=True)

In [15]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9973 entries, 59 to 22635
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   NAME                9973 non-null   object 
 1   FOLLOWERS           9973 non-null   float64
 2   ER                  9973 non-null   float64
 3   COUNTRY             9837 non-null   object 
 4   TOPIC OF INFLUENCE  9973 non-null   object 
 5   POTENTIAL REACH     9973 non-null   float64
 6   CHANNEL             9973 non-null   object 
dtypes: float64(3), object(4)
memory usage: 623.3+ KB


In [16]:
#  final_df.iloc[90:121]

In [17]:
# Standardize the text in the 'Topic of Influence' column
def standardize_topic(topic):
    """Standardizes the text in the 'Topic of Influence' column."""
    topic = topic.lower()
    topic = ''.join(ch for ch in topic if ch.isalnum() or ch.isspace())
    topic = ' '.join(topic.split())
    return topic

final_df['TOPIC OF INFLUENCE'] = final_df['TOPIC OF INFLUENCE'].astype(str).apply(standardize_topic)

# Print the unique values in the 'TOPIC OF INFLUENCE' column
print("Unique values in 'TOPIC OF INFLUENCE' column after standardization:")
print(final_df['TOPIC OF INFLUENCE'].unique())

Unique values in 'TOPIC OF INFLUENCE' column after standardization:
['celebrity' 'cricket sports' 'art' ...
 'nature outdoor activity fitness health education' 'finance politics'
 'entertainment music family romance wedding education']


In [18]:
duplicate_values = final_df['NAME'].duplicated().sum()
print(duplicate_values)

851


In [19]:
# Drop duplicates based on Name column
final_df = final_df.drop_duplicates(subset=['NAME'])


In [20]:
# Save the DataFrame to a CSV file
final_df.to_csv('merged_influencer_data.csv', index=False) 

In [21]:

# Load the dataset
merged_df = pd.read_csv('merged_influencer_data.csv')

# Split the 'NAME' column at '@', but only if '@' is present
merged_df[['NAME', 'USERNAME']] = merged_df['NAME'].str.split('@', n=1, expand=True)

# Handle cases where '@' is not found (USERNAME will be None)
merged_df['USERNAME'] = merged_df['USERNAME'].fillna('')

# Remove leading whitespace from 'USERNAME' (if any)
merged_df['USERNAME'] = merged_df['USERNAME'].str.lstrip()

# Get a list of columns
cols = list(merged_df.columns)

# Move 'USERNAME' to the index after 'NAME'
cols.insert(cols.index('NAME') + 1, cols.pop(cols.index('USERNAME')))

# Reorder the DataFrame columns
merged_df = merged_df[cols]


merged_df.head()

,NAME,USERNAME,FOLLOWERS,ER,COUNTRY,TOPIC OF INFLUENCE,POTENTIAL REACH,CHANNEL
0,Pashto Entertainment Entertainment,UC5o5At6pto0EYYa0-m3GIiA,19400.0,1.10,Afghanistan,celebrity,5800.0,youtube
1,Rashid Khan,rashid.khan19,9100000.0,6.19,Afghanistan,cricket sports,2700000.0,instagram
2,Steve McCurry,stevemccurryofficial,3600000.0,1.07,Afghanistan,art,1100000.0,instagram
3,د. وسيم يوسف,waseem_yousef,3300000.0,0.19,Afghanistan,celebrities,993100.0,instagram
4,AHMAD SAEEDI / احمد سعيدي,ahmadsaeedi1,3200000.0,1.71,Afghanistan,music singer songwriting,948300.0,instagram


In [22]:
# # Find rows where 'USERNAME' is 'some_username'
# matching_rows = merged_df.query('USERNAME == "britneyspears"')
# print(matching_rows)


In [23]:
# Save the updated DataFrame
merged_df.to_csv('merged_influencer_data.csv', index=False)

In [24]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9122 entries, 0 to 9121
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   NAME                9122 non-null   object 
 1   USERNAME            9122 non-null   object 
 2   FOLLOWERS           9122 non-null   float64
 3   ER                  9122 non-null   float64
 4   COUNTRY             9047 non-null   object 
 5   TOPIC OF INFLUENCE  9122 non-null   object 
 6   POTENTIAL REACH     9122 non-null   float64
 7   CHANNEL             9122 non-null   object 
dtypes: float64(3), object(5)
memory usage: 570.3+ KB


In [1]:
pip install kaggle



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install opendatasets


In [7]:
pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 4.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
Note: you may need to restart the kernel to use updated packages.


In [10]:
import opendatasets as od 


od.download( "https://www.kaggle.com/datasets/syedjaferk/top-1000-instagrammers-world-cleaned/data") 


Dataset URL: https://www.kaggle.com/datasets/syedjaferk/top-1000-instagrammers-world-cleaned


100%|██████████████████████████████████████| 21.5k/21.5k [00:00<00:00, 3.02MB/s]

In [11]:
od.download("https://www.kaggle.com/datasets/faisaljanjua0555/top-200-most-followed-instagram-accounts-2023")

Dataset URL: https://www.kaggle.com/datasets/faisaljanjua0555/top-200-most-followed-instagram-accounts-2023


100%|██████████████████████████████████████| 4.67k/4.67k [00:00<00:00, 1.99MB/s]

In [2]:
pip install nbconvert

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandoc

  Preparing metadata (setup.py) ... done
  Created wheel for pandoc: filename=pandoc-2.4-py3-none-any.whl size=34794 sha256=ca1fbee1fa6a3606457c89d2efde8733d4f47f274f9470fcd2069ea39a6de661
  Stored in directory: /Users/flavialooker/Library/Caches/pip/wheels/4f/d7/32/c6c9b7b05e852e920fd72174487be3a0f18e633a7adcc303be
Successfully built pandoc
Note: you may need to restart the kernel to use updated packages.
